In [3]:
#!pip install openai
#!pip install pydub
#!pip install sounddevice

In [4]:
from openai import OpenAI
openai = OpenAI(api_key="input your API Key here")

In [5]:
# import openai # commented out so you don't import again and have to set your api key in every cell
# openai.api_key = "YOUR API KEY HERE" # commented out so you don't overwrite the correct api key you set above

def get_response(messages):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=300
    )
    content = response.choices[0].message.content
    return content


def chat():
    messages = [
        {"role": "system", "content": "You are a friendly assistant"} # change me to change my personality
    ]
    while True:
        prompt = input("Type a prompt (or type \"exit\" to exit)...")
        print("User:", prompt)
        if prompt == "exit":
            break
        messages = add_message(messages, prompt, "user")
        response = get_response(messages)
        messages = add_message(messages, response, "assistant")
        print("Assistant:", response)


def add_message(messages, content, role):
    message = {"role": role, "content": content}
    messages.append(message)
    return messages


chat()


User: 
Assistant: Hello! How can I assist you today?
User: hi
Assistant: Hello! How are you doing today?
User: what can i buy for my 9 year old niece
Assistant: That's great that you're thinking of getting something for your 9-year-old niece! How about considering some options like art supplies, books, puzzles, board games, craft kits, educational toys, or perhaps a fun accessory like a new backpack or jewelry? You could also consider her interests and hobbies to find something she would really enjoy. Do you have any specific ideas in mind yet?
User: exit


Voice Generation OpenAI has recently introduced text-to-speech capabilities to their platform with the launch of a new text-to-speech model. You can explore the documentation on how to use it here.

Before integrating voice generation into our chat system, let's first develop a function that converts text into speech and then plays the audio.

In [6]:
from IPython.display import Audio, display


def say(text):
    speech_file_path = "speech.mp3"
    response = openai.audio.speech.create( # TODO call the function to create speech
        model="tts-1", # TODO specify the model
        voice="alloy", # TODO specify the voice
        input=text # TODO specify the input text
    )
    response.stream_to_file(speech_file_path) # TODO stream the response to the file path
    #  print(dir(response)) # you can always use the dir() function to see what methods are available on an object (this isn't yet in the docs)
    display(Audio(speech_file_path, autoplay=True)) # display the audio file (this would be different if we were working in a .py file instead of a .ipynb notebook file)

say("Hello, my name is Jarvis. I am a robot.") # test the function

C:\Users\Emeka Ichoku\AppData\Local\Temp\ipykernel_10396\1211849703.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path) # TODO stream the response to the file path


In [7]:
# import openai # commented out so you don't import again and have to set your api key in every cell
# openai.api_key = "YOUR API KEY HERE" # commented out so you don't overwrite the correct api key you set above

from time import sleep # needed for random colab-only reason (see usage and commentbelow)

def get_response(messages):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=300
    )
    content = response.choices[0].message.content
    return content


def chat():
    messages = [
        {"role": "system", "content": "You are a friendly assistant"} # change me to change my personality
    ]
    while True:
        prompt = input("Type a prompt (or type \"exit\" to exit)...")
        print("User:", prompt)
        if prompt == "exit":
            break
        messages = add_message(messages, prompt, "user")
        response = get_response(messages)
        messages = add_message(messages, response, "assistant")
        print("Assistant:", response) # TODO (optional) remove this line that prints the response
        say(response) # TODO call the say() function with the response
        sleep(1) # seemed necessary to wait for 1 second to render both the audio player and the text input on colab


def add_message(messages, content, role):
    message = {"role": role, "content": content}
    messages.append(message)
    return messages


chat()


User: what can i get for my 9 year old niece
Assistant: That's a wonderful question! There are so many great options for a 9-year-old niece. Some ideas could include:

1. Books: Consider age-appropriate books that match her interests.

2. Art supplies: Crayons, colored pencils, markers, and sketchbooks are always a hit.

3. Puzzles and games: A fun puzzle or board game can provide hours of entertainment.

4. Craft kits: DIY craft kits can spark creativity and keep her busy.

5. Toys: Depending on her interests, dolls, action figures, or building sets can be great options.

6. Educational gifts: Science kits, coding kits, or educational games can be both fun and educational.

7. Outdoor toys: Consider sports equipment, a bike, or a kite for outdoor fun.

Remember to consider her interests and hobbies when choosing a gift. I hope these ideas help you find the perfect present for your niece!


C:\Users\Emeka Ichoku\AppData\Local\Temp\ipykernel_10396\1211849703.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path) # TODO stream the response to the file path


User: exit


Voice Transcription
The subsequent phase involves enabling our AI system to listen to our voice and convert it into text for processing.

In [8]:
from time import time
from pydub import AudioSegment
import sounddevice as sd
import numpy as np
import os

def voice_to_text(duration=10):
    print("Listening...")

    # RECORD
    # this section is a little complicated, but I'm fairly sure I found it on stackoverflow
    fs = 44100  # Sample rate (samples per second)
    audio_data = []
    with sd.InputStream(samplerate=fs, channels=1, dtype=np.int16) as stream:
        start_time = time()
        while time() - start_time < duration:
            # Read a chunk of audio data
            chunk, overflowed = stream.read(fs)
            audio_data.append(chunk)
    audio_data = np.concatenate(audio_data) # Convert the recorded audio data into a NumPy array
    audio = AudioSegment(audio_data.tobytes(), frame_rate=fs, sample_width=2, channels=1) # this line gave me loads of trouble but eventually these params worked
    temp_audio_filename = "temp.mp3"
    audio.export(temp_audio_filename, format="mp3")

    # TRANSCRIBE
    with open(temp_audio_filename, 'rb') as f: # TODO open context manager to temporary audio file
        print("Thinking...")
        text = openai.audio.transcriptions.create( # TODO call the function to create a transcription
            model="whisper-1", # TODO specify the model
            file=f # TODO specify the file
        ).text # TODO get the text attribute from the response

    os.remove(temp_audio_filename) # TODO remove the temporary audio file

    return text

voice_to_text()

Listening...
Thinking...


'Oh, can I get gum too?'

adding this into our main body of code.

In [11]:
# import openai # commented out so you don't import again and have to set your api key in every cell
# openai.api_key = "YOUR API KEY HERE" # commented out so you don't overwrite the correct api key you set above

# your system message probably has a lot of text in that you would need to copy over, so I haven't included it down here


def get_response(messages):
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=300
    )
    content = response.choices[0].message.content
    return content


def chat():
    messages = [
        {"role": "system", "content": "You are a friendly assistant"} # change me to change my personality
    ]
    while True:
        # prompt = input("Type a prompt (or type \"exit\" to exit)...") # TODO remove this line as our input will now be taken by voice
        prompt = voice_to_text()
        print("User:", prompt)
        if prompt == "Bye Jarvis.": # TODO change this to check if the prompt is "Exit." because this is the format that whisper transcribes "exit" to (try without first)
            break
        messages = add_message(messages, prompt, "user")
        response = get_response(messages)
        messages = add_message(messages, response, "assistant")
        # print("Assistant:", response) 
        say(response)
        input("Press enter to continue") # TODO add a line here to wait until the user presses enter to continue (try without this first and you'll see why we need it)


def add_message(messages, content, role):
    message = {"role": role, "content": content}
    messages.append(message)
    return messages


chat()

Listening...
Thinking...
User: Bye Jarvis.
